# Identifying Liquidity Patterns
Analyze the bid-ask spread and order book depth in real time to help traders decide when to execute large trades with minimal price impact.

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd

# Function to analyze bid-ask spread
def analyze_bid_ask_spread(order_book):
    """
    Analyzes the bid-ask spread from the order book.
    
    Parameters:
    order_book (pd.DataFrame): DataFrame containing order book data with 'bid_price', 'ask_price', 'bid_size', 'ask_size' columns.
    
    Returns:
    float: The bid-ask spread.
    """
    best_bid = order_book['bid_price'].max()
    best_ask = order_book['ask_price'].min()
    spread = best_ask - best_bid
    return spread

# Function to analyze order book depth
def analyze_order_book_depth(order_book, depth=10):
    """
    Analyzes the order book depth.
    
    Parameters:
    order_book (pd.DataFrame): DataFrame containing order book data with 'bid_price', 'ask_price', 'bid_size', 'ask_size' columns.
    depth (int): The number of levels to consider for depth analysis.
    
    Returns:
    dict: A dictionary with total bid and ask sizes up to the specified depth.
    """
    sorted_bids = order_book.sort_values(by='bid_price', ascending=False).head(depth)
    sorted_asks = order_book.sort_values(by='ask_price', ascending=True).head(depth)
    
    total_bid_size = sorted_bids['bid_size'].sum()
    total_ask_size = sorted_asks['ask_size'].sum()
    
    return {'total_bid_size': total_bid_size, 'total_ask_size': total_ask_size}

# Example usage
order_book_data = {
    'bid_price': [100, 99, 98, 97, 96],
    'ask_price': [101, 102, 103, 104, 105],
    'bid_size': [10, 15, 10, 5, 2],
    'ask_size': [8, 12, 15, 10, 5]
}
order_book = pd.DataFrame(order_book_data)

# Analyze bid-ask spread
spread = analyze_bid_ask_spread(order_book)
print(f"Bid-Ask Spread: {spread}")

# Analyze order book depth
depth_analysis = analyze_order_book_depth(order_book, depth=3)
print(f"Order Book Depth: {depth_analysis}")

Bid-Ask Spread: 1
Order Book Depth: {'total_bid_size': np.int64(35), 'total_ask_size': np.int64(35)}


# Detecting Large Institutional Orders (Iceberg Orders)
Track order book changes to detect suspiciously large trades broken into small parts, helping traders anticipate big moves before they happen.

In [3]:
# Function to detect iceberg orders
def detect_iceberg_orders(order_book, threshold=100):
    """
    Detects iceberg orders by tracking suspiciously large trades broken into small parts.
    
    Parameters:
    order_book (pd.DataFrame): DataFrame containing order book data with 'bid_price', 'ask_price', 'bid_size', 'ask_size' columns.
    threshold (int): The size threshold to consider for detecting iceberg orders.
    
    Returns:
    pd.DataFrame: DataFrame containing detected iceberg orders.
    """
    iceberg_orders = order_book[(order_book['bid_size'] > threshold) | (order_book['ask_size'] > threshold)]
    return iceberg_orders

# Example usage
iceberg_orders = detect_iceberg_orders(order_book, threshold=10)
print("Detected Iceberg Orders:")
print(iceberg_orders)

Detected Iceberg Orders:
   bid_price  ask_price  bid_size  ask_size
1         99        102        15        12
2         98        103        10        15


# Predicting Market Impact of Large Trades
Predict how much price movement will occur before executing a trade to help minimize slippage.

In [ ]:
# Function to predict market impact of large trades
def predict_market_impact(order_book, trade_size):
    """
    Predicts the market impact of a large trade.
    
    Parameters:
    order_book (pd.DataFrame): DataFrame containing order book data with 'bid_price', 'ask_price', 'bid_size', 'ask_size' columns.
    trade_size (int): The size of the trade to be executed.
    
    Returns:
    float: The predicted price impact.
    """
    # Calculate the total market depth
    total_bid_size = order_book['bid_size'].sum()
    total_ask_size = order_book['ask_size'].sum()
    
    # Calculate the average bid and ask prices
    avg_bid_price = (order_book['bid_price'] * order_book['bid_size']).sum() / total_bid_size
    avg_ask_price = (order_book['ask_price'] * order_book['ask_size']).sum() / total_ask_size
    
    # Estimate the price impact
    if trade_size > total_bid_size:
        price_impact = avg_ask_price - avg_bid_price
    else:
        price_impact = (avg_ask_price - avg_bid_price) * (trade_size / total_bid_size)
    
    return price_impact

# Example usage
trade_size = 20
price_impact = predict_market_impact(order_book, trade_size)
print(f"Predicted Price Impact for trade size {trade_size}: {price_impact}")

# Real-Time Arbitrage Opportunities
Monitor multiple order books to find price mismatches, allowing traders to execute risk-free arbitrage.

In [ ]:
# Function to find real-time arbitrage opportunities
def find_arbitrage_opportunities(order_books):
    """
    Finds real-time arbitrage opportunities across multiple exchanges.
    
    Parameters:
    order_books (dict): Dictionary containing order book data for multiple exchanges. 
                        Keys are exchange names, values are DataFrames with 'bid_price', 'ask_price', 'bid_size', 'ask_size' columns.
    
    Returns:
    list: A list of arbitrage opportunities with details.
    """
    opportunities = []
    
    # Iterate through each pair of exchanges
    exchanges = list(order_books.keys())
    for i in range(len(exchanges)):
        for j in range(i + 1, len(exchanges)):
            exchange_a = exchanges[i]
            exchange_b = exchanges[j]
            
            order_book_a = order_books[exchange_a]
            order_book_b = order_books[exchange_b]
            
            # Find the best bid and ask prices for both exchanges
            best_bid_a = order_book_a['bid_price'].max()
            best_ask_a = order_book_a['ask_price'].min()
            best_bid_b = order_book_b['bid_price'].max()
            best_ask_b = order_book_b['ask_price'].min()
            
            # Check for arbitrage opportunity: Buy on one exchange and sell on another
            if best_bid_a > best_ask_b:
                profit = best_bid_a - best_ask_b
                opportunities.append({
                    'buy_exchange': exchange_b,
                    'sell_exchange': exchange_a,
                    'buy_price': best_ask_b,
                    'sell_price': best_bid_a,
                    'profit': profit
                })
            elif best_bid_b > best_ask_a:
                profit = best_bid_b - best_ask_a
                opportunities.append({
                    'buy_exchange': exchange_a,
                    'sell_exchange': exchange_b,
                    'buy_price': best_ask_a,
                    'sell_price': best_bid_b,
                    'profit': profit
                })
    
    return opportunities

# Example usage
order_books = {
    'Exchange_A': pd.DataFrame({
        'bid_price': [100, 99, 98, 97, 96],
        'ask_price': [101, 102, 103, 104, 105],
        'bid_size': [10, 15, 10, 5, 2],
        'ask_size': [8, 12, 15, 10, 5]
    }),
    'Exchange_B': pd.DataFrame({
        'bid_price': [101, 100, 99, 98, 97],
        'ask_price': [102, 103, 104, 105, 106],
        'bid_size': [12, 10, 8, 5, 3],
        'ask_size': [10, 15, 10, 5, 2]
    })
}

# Find arbitrage opportunities
arbitrage_opportunities = find_arbitrage_opportunities(order_books)
print("Arbitrage Opportunities:")
for opportunity in arbitrage_opportunities:
    print(opportunity)

# Optimizing Market-Making Strategies
Analyze order book pressure to help market makers adjust spreads dynamically based on order flow.

In [ ]:
# Function to optimize market-making strategies
def optimize_market_making(order_book):
    """
    Optimizes market-making strategies by analyzing order book pressure.
    
    Parameters:
    order_book (pd.DataFrame): DataFrame containing order book data with 'bid_price', 'ask_price', 'bid_size', 'ask_size' columns.
    
    Returns:
    dict: A dictionary with suggested bid-ask spread adjustments.
    """
    # Calculate total bid and ask sizes
    total_bid_size = order_book['bid_size'].sum()
    total_ask_size = order_book['ask_size'].sum()
    
    # Calculate order book pressure
    pressure = total_bid_size - total_ask_size
    
    # Suggest spread adjustments based on pressure
    if pressure > 0:
        suggestion = {
            'action': 'narrow',
            'reason': 'More buy orders, suggesting upward pressure on prices.'
        }
    else:
        suggestion = {
            'action': 'widen',
            'reason': 'More sell orders, suggesting downward pressure on prices.'
        }
    
    return suggestion

# Example usage
market_making_suggestion = optimize_market_making(order_book)
print("Market-Making Strategy Suggestion:")
print(market_making_suggestion)